In [1]:
import docplex
import pandas as pd
import tensorflow as tf
import numpy as np
import utility
import copy
import mlp_explainer
import mymetrics
import time
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets
from sklearn.model_selection import train_test_split
from milp import codify_network
from teste import get_minimal_explanation
from sklearn.metrics import classification_report

In [4]:
# Load Dataset
dataset_name = 'Iris' #Iris, Wine, Vertebral-Column, Pima, Parkinsons, Breast_Cancer, Blood_Transfusion, Ionosphere, Glass, Climate, Modeling, Banknote, Sonar
#df_artificial = pd.read_csv('datasets/artificial/'+f'{dataset_name}_artificial.csv')
loaded_bounds = np.load(f'bounds/{dataset_name}_data_bounds.npz')
original_bounds = loaded_bounds['original_bounds']
keras_model = tf.keras.models.load_model(f'new_models/{dataset_name}.h5')
mp_model, output_bounds = #Load this from pickle
print(original_bounds)
np.random.seed(50)

[[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [5]:
d = 0.1
p_value = 0.25
X_test = pd.read_csv(f'{dataset_name}_results/{dataset_name}_X_test.csv')
onestep_coverages = []
twostep_coverages = []

In [ ]:
for instance in X_test.values[:2]:
    #bounds based on original_bounds (for artificial)    
    
    twostep_coverages = []
    twostep_explanations = []
    for j in range(len(pred_dataset_df['target'].unique())):
        for i, sample in enumerate((testing_data.to_numpy()[:, :-1][class_indexes[j]])):  
            explanation, minimal = mlp_explainer.run_explanation_doublestep(sample = sample, n_classes=n_classes, kmodel=keras_model, model=mp_model, output_bounds=output_bounds, og_bounds=original_bounds, p=p_value)
            twostep_explanations.append(explanation)
            twostep_coverages.append(len(mymetrics.calculate_coverage(testing_data, twostep_explanations[-1])))
    
            explanation, minimal = mlp_explainer.run_explanation(sample = sample, n_classes=n_classes, kmodel=keras_model, model=mp_model, output_bounds=output_bounds, og_bounds=original_bounds, enable_log=False,                                                  )
            onestep_explanations.append(explanation)
            onestep_coverages.append(len(mymetrics.calculate_coverage(testing_data, onestep_explanations[-1])))
        
def compute_mean_std(arr):
    return np.mean(arr), np.std(arr)

def relative_percentage_diff(new, old):
    if np.any(old == 0):
        print(f'Warning: found possible division by zero')
        return np.where(old != 0, ((new - old) / old) * 100, np.nan)
    return ((new - old) / old) * 100
    
onestep_coverages = np.array(onestep_coverages)
twostep_coverages = np.array(twostep_coverages)

(coverage_mean_onestep, coverage_std_onestep) = compute_mean_std(onestep_coverages)
(coverage_mean_twostep, coverage_std_twostep) = compute_mean_std(twostep_coverages)

coverage_mean_diff = relative_percentage_diff(coverage_mean_twostep, coverage_mean_onestep)
coverage_std_diff = relative_percentage_diff(coverage_std_twostep, coverage_std_onestep)

coverage_relative_pointwise = relative_percentage_diff(twostep_coverages, onestep_coverages)
coverage_relative_mean = np.mean(coverage_relative_pointwise)
coverage_relative_std = np.std(coverage_relative_pointwise)

# Organize Data
all_metrics_data = {
    'Metric': ['Coverage'],
    'ONESTEP_MEAN': [coverage_mean_onestep],
    'ONESTEP_STD': [coverage_std_onestep],
    'TWOSTEP_MEAN': [coverage_mean_twostep],
    'TWOSTEP_STD': [coverage_std_twostep],
    'MEAN_DIFF_%': [coverage_mean_diff],
    'STD_DIFF_%': [coverage_std_diff],
    'POINTWISE_MEAN_%': [coverage_relative_mean],
    'POINTWISE_STD_%': [coverage_relative_std]
}
# Display and save
all_metrics_df = pd.DataFrame(all_metrics_data)
display(all_metrics_df)
#all_metrics_df.to_csv(f'{dataset_name}_results/coverage_results_artificial_{p}.csv', index=False)

#Save Raw Metric Data
raw_df = pd.DataFrame({
    "onestep_coverages": onestep_coverages, 
    "twostep_coverages": twostep_coverages,
    "coverage_relative_%": coverage_relative_pointwise
})

display(raw_df)
#raw_df.to_csv(f"{dataset_name}_results/coverage_raw_metric_data_{p}.csv", index=False)